In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
import pandas as pd
import tensorflow as tf
import numpy as np
Sequential = tf.keras.models.Sequential
TextVectorization = tf.keras.layers.TextVectorization
Embedding = tf.keras.layers.Embedding
GlobalAveragePooling1D = tf.keras.layers.GlobalAveragePooling1D
Dense = tf.keras.layers.Dense
print(f"TensorFlow is ready: {tf.__version__}")

In [ ]:
train_df = pd.read_csv(train_file_path, sep='\t', header=None, names=['label', 'message'])
test_df = pd.read_csv(test_file_path, sep='\t', header=None, names=['label', 'message'])
train_labels = train_df['label'].map({'ham': 0, 'spam': 1}).values.astype('float32')
test_labels = test_df['label'].map({'ham': 0, 'spam': 1}).values.astype('float32')

In [ ]:
MAX_WORDS = 1000
MAX_LEN = 100
vectorizer = TextVectorization(max_tokens=MAX_WORDS, output_sequence_length=MAX_LEN)
vectorizer.adapt(train_df['message'].values)
model = tf.keras.Sequential([
    vectorizer,
    tf.keras.layers.Embedding(1000, 16),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(train_df['message'].values, train_labels, epochs=10, verbose=1)

In [ ]:
def predict_message(pred_text):
    text_input = np.array([pred_text], dtype=object)
    prediction = model(text_input, training=False)
    prob = prediction.numpy()[0][0]
    label = "spam" if prob >= 0.5 else "ham"
    return [float(prob), label]

In [ ]:
def test_predictions():
    test_messages = ["how are you doing today", "sale today! to stop texts call 98912460324"]
    test_answers = ["ham", "spam"]
    passed = True
    for msg, ans in zip(test_messages, test_answers):
        prediction = predict_message(msg)
        if prediction[1] != ans: passed = False
    if passed: print("You passed the challenge!")
    else: print("Keep trying.")
test_predictions()